In [1]:
import nltk

from heapq import nlargest

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np


In [2]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to C:\Users\Sergiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergiy/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

# Сумаризація тексту за допомогою `nltk`

1. [Суммаризация текста: подходы, алгоритмы, рекомендации и перспективы](https://habr.com/ru/articles/514540/)

Скористаємось алгоритмом екстрактивної сумаризації запропонованим в статті [1]

- Розбиття вхідного тексту на окремі речення 
- Переведення речення у цифрове представлення (вектор).
- Обчислення і збереження в матриці подібності подібності між векторами речень.
- Перетворення отриманої матриці на граф із реченнями у вигляді вершин і оцінками подібності у вигляді ребер для обчислення рангу речень.
- Вибір пропозицій з найвищою оцінкою для підсумкового резюме.

## Створюємо теку з українськими стоп-словами

In [3]:
import os
import requests

# Перевіряємо наявність папки corpora/stopwords у каталозі nltk_data
nltk_data_path = nltk.data.path[0]
stopwords_path = os.path.join(nltk_data_path, "corpora", "stopwords")
if not os.path.exists(stopwords_path):
    os.makedirs(stopwords_path)

# Завантажуємо стоп-слова для української мови та зберігаємо у файл
url = (
    "https://raw.githubusercontent.com/olegdubetcky/Ukrainian-Stopwords/main/ukrainian"
)
r = requests.get(url)

with open(os.path.join(stopwords_path, "ukrainian"), "wb") as f:
    f.write(r.content)

# Додаємо українські стоп-слова в NLTK
stop_words = set(stopwords.words("ukrainian"))

## Завантажуємо текст з файлу у змінну

In [4]:
with open("text_ua.txt", "r", encoding="utf-8") as file:
    text = file.read()

## Визначаємо функції

1. [Cosine Similarity and Cosine Distance](https://medium.com/geekculture/cosine-similarity-and-cosine-distance-48eed889a5c4)

In [5]:
def sentence_similarity(sent1, sent2, stopwords=None):
    """
    Функція вимірює схожість між двома реченнями. Використовується косинусна відстань між векторами, що представляють слова в реченнях. Вона також враховує стоп-слова.
    """
    if stopwords is None:
        stopwords = set()

    # Токенізуємо речення та видаляємо стоп-слова

    words1 = [
        word.lower()
        for word in word_tokenize(sent1)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення
    words2 = [
        word.lower()
        for word in word_tokenize(sent2)
        if word.isalnum() and word.lower() not in stopwords
    ]  # створюємо список слів (без стоп-слів) із першого речення

    all_words = list(
        set(words1 + words2)
    )  # створюємо список унікальних слів з двох речень

    # Створюємо вектори типу компонентами яких є  одиниці та нулі
    # якщо слово є в речення, то ставимо 1, інакше 0
    # Отримуємо щось таке [1, 0, 0, 1, ...]
    vector1 = [1 if word in words1 else 0 for word in all_words]
    vector2 = [1 if word in words2 else 0 for word in all_words]

    # розраховуємо косинусну відстань між двома векторами

    return 1 - cosine_distance(vector1, vector2)


def build_similarity_matrix(sentences, stop_words):
    """
    Функція будує матрицю схожості між усіма парами речень у тексті.
    """
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = sentence_similarity(
                    sentences[i], sentences[j], stop_words
                )

    return similarity_matrix


def generate_summary(text, num_sentences=3, stop_words=stop_words):
    """
    Функція створює короткий зміст тексту.

    Алгоритм роботи:

    1. Читаємо речення із тексту і потім будуємо матрицю схожості.
    2. Розраховуємо суму схожості для кожного речення.
    3. Використовуємо nlargest для вибору топ N речень за порядком зменшення схожості.
    4. Об'єднуємо вибрані речення в один рядок.

    """
    summarize_text = []

    sentences = sent_tokenize(text)  # Токенізуємо текст

    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    sentence_similarity_scores = np.array(
        [sum(row) for row in sentence_similarity_matrix]
    )

    # Використовуємо nlargest для вибору топ N речень за порядком зменшення схожості
    top_sentences_indices = nlargest(
        num_sentences,
        range(len(sentence_similarity_scores)),
        key=sentence_similarity_scores.__getitem__,
    )

    # Формуємо короткий зміст з вибраних речень
    for i in top_sentences_indices:
        summarize_text.append(sentences[i])

    return " ".join(summarize_text)

## Записуємо анотований текст до файлу

In [6]:
num_sentences = 1
stop_words = set(stopwords.words("ukrainian"))
summary = generate_summary(text, num_sentences, stop_words)
with open("summary_ua.txt", "w", encoding="utf-8") as file:
    file.write(summary)


# Висновки

1. [Вихідний текст](./text_ua.txt)
2. [Анотація](./summary_ua.txt)

In [7]:
print(f"Довжина вихідного тексту: {len(text)}")
print(f"Довжина анотації: {len(summary)}")

Довжина вихідного тексту: 3110
Довжина анотації: 252


In [9]:
print(
    f"Таким чином текст довжиною {len(text)} слів зменшився до {len(summary)} слів для {num_sentences} речень."
)


Таким чином текст довжиною 3110 слів зменшився до 252 слів для 1 речень.
